# TIF360 Project

Main source: https://www.kaggle.com/code/rmonge/predicting-molecule-properties-based-on-its-smiles/notebook

### Import packages

In [50]:
import os
import rdkit
from rdkit import Chem  # To extract information of the molecules
from rdkit.Chem import Draw  # To draw the molecules
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random

import torch
import torch_geometric
from torch_geometric.loader import DataLoader
import torch_geometric.utils as utils
import networkx as nx
from torch.nn import Linear
from torch_geometric.nn import global_mean_pool, GraphConv, GATConv, GCNConv, BatchNorm, GraphNorm
import torch.nn.functional as F

from sklearn.metrics import r2_score

### Load data

In [101]:
df = pd.read_csv("../data/smiles_and_targets.csv")
print(np.shape(df))

(132820, 21)


### Load descriptors

In [138]:
mol_descriptor = np.load("../data/Mordred_mol_descriptors.npy")
print(mol_descriptor.shape)

(132820, 987)


### Load Morgan fingerprints

In [103]:
mol_fingerprints = np.load("../data/mol_morgan_fingerprints.npy")
print(mol_fingerprints.shape)

(132820, 2048)


# Investigate Neural Networks

## Graph Neural Networks

#### Convert data to graphs

In [104]:
# import packages
import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader

In [105]:
from graph_dataset_functions import create_graph_dataset_from_smiles

properties_names = ['A', 'B', 'C', 'mu', 'alfa', 'homo', 'lumo', 'gap', 'R²', 'zpve', 'U0', 'U', 'H', 'G', 'Cv']

x_smiles = df.smiles.values
y = df.loc[:, properties_names].values  # shape = (n_samples, n_properties)

dataset = create_graph_dataset_from_smiles(x_smiles, y[0:len(x_smiles), :])

Information of the graph dataset

In [106]:
print(f'Number of graphs (molecules): {len(dataset)}')
graph = dataset[50]
print('=================================================================================')
print(f'Properties of graph {50}, molecule smiles: {df.smiles.values[50]}')
print(f'Number of nodes: {dataset[50].x.shape[0]}')
print(f'Number of edges: {dataset[50].edge_index.shape[1]}')
print(f'Number of node features: {dataset[50].x.shape[1]}')
print(f'Number of edge features: {dataset[50].edge_attr.shape[1]}')
print(f'Number of target properties: {dataset[50].y.shape[1]}')

Number of graphs (molecules): 132820
Properties of graph 50, molecule smiles: CC1=CNC=C1
Number of nodes: 6
Number of edges: 12
Number of node features: 78
Number of edge features: 10
Number of target properties: 15


Filterout data with no edge features defined (Like ex: CH4) (These causes problems down the line)

In [107]:
indexes_to_delete = []
for item in range(0,len(dataset)):
    if dataset[item].edge_attr.shape[1] == 0:
        indexes_to_delete.append(item)
    else:
        pass

indexes_to_delete.sort()

print("Number of none edge feature molecules: ", len(indexes_to_delete))

print("Before: ", len(dataset))

for item in range(0,len(indexes_to_delete)):
    print("Molecule to delete: ", df.smiles.values[indexes_to_delete[item]])
    #del dataset[indexes_to_delete[item] - item] 
    dataset.pop((indexes_to_delete[item] - item)) # -item since all future data points will have its index reduced by 1 for each deleted previous data point
print("After: ", len(dataset))

Number of none edge feature molecules:  0
Before:  132820
After:  132820


Split data into train and test set

In [108]:
from torch_geometric.loader import DataLoader
from sklearn.preprocessing import StandardScaler, MinMaxScaler

# split the dataset into test and validation:
num_samples = len(dataset)

# Want to divide data randomly
random.seed(42)
random_indexes = np.array(random.sample(range(num_samples), num_samples)) # random.sample ensures no duplicates

train_data = [dataset[index] for index in random_indexes[int(.2 * num_samples ) :]] # 80%
test_data = [dataset[index] for index in random_indexes[: int(.2 * num_samples )]] # 20%

print("Example train data target before scaling", train_data[0].y)
train_data_targets = [data.y for data in train_data]
train_data_targets = torch.concatenate(train_data_targets, axis=0)
test_data_targets = [data.y for data in test_data]
test_data_targets = torch.concatenate(test_data_targets, axis=0)

scaler = StandardScaler()
train_data_targets = scaler.fit_transform(train_data_targets)
test_data_targets = scaler.transform(test_data_targets)

# print("Example train data target", train_data_targets[0].reshape(1,-1).shape)
train_data_targets = torch.tensor(train_data_targets, dtype=torch.float)
test_data_targets = torch.tensor(test_data_targets, dtype=torch.float)

train_data = [Data(x=data.x, edge_index=data.edge_index, edge_attr=data.edge_attr, y=train_data_targets[index].reshape(1,-1)) for index, data in enumerate(train_data)]
test_data = [Data(x=data.x, edge_index=data.edge_index, edge_attr=data.edge_attr, y=test_data_targets[index].reshape(1,-1)) for index, data in enumerate(test_data)]
print("Example train data target after scaling:", train_data[0].y)

print("Total data size: ", len(dataset))
print("Train data size: ", len(train_data))
print("Test data size: ", len(test_data))

#train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
#test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

Example train data target before scaling tensor([[ 2.9693e+00,  1.9777e+00,  1.8423e+00,  1.2093e+00,  8.4060e+01,
         -1.8970e-01, -7.0400e-02,  1.1930e-01,  8.7339e+02,  1.5988e-01,
         -3.4861e+02, -3.4860e+02, -3.4860e+02, -3.4864e+02,  2.9189e+01]])
Example train data target after scaling: tensor([[-0.3350,  1.3300,  2.2841, -1.0264,  1.0990,  2.2975, -1.7445, -2.7966,
         -1.1527,  0.3357,  1.6155,  1.6155,  1.6155,  1.6156, -0.6067]])
Total data size:  132820
Train data size:  106256
Test data size:  26564


Divide descriptors and fingerprints into train and test

In [139]:
train_data_descriptor = [mol_descriptor[index,:] for index in random_indexes[int(.2 * num_samples ) :]] # 80%
test_data_desriptors = [mol_descriptor[index,:] for index in random_indexes[: int(.2 * num_samples )]] # 20%

train_data_fingerprints = [mol_fingerprints[index,:] for index in random_indexes[int(.2 * num_samples ) :]] # 80%
test_data_fingerprints = [mol_fingerprints[index,:] for index in random_indexes[: int(.2 * num_samples )]] # 20%


Normalize using MinMax

In [140]:
minmax_scaler = MinMaxScaler()

train_data_descriptor = minmax_scaler.fit_transform(train_data_descriptor)

test_data_desriptors = minmax_scaler.transform(test_data_desriptors)

In [141]:
minmax_scaler = MinMaxScaler()

train_data_fingerprints = minmax_scaler.fit_transform(train_data_fingerprints)

test_data_fingerprints = minmax_scaler.transform(test_data_fingerprints)

Combine descriptors temporarely with in train and test data for graphs

In [112]:
#Testing stuff

temp = train_data[1000]
temp.descriptors = train_data_descriptor[1000,:]

print(temp)

Data(x=[9, 78], edge_index=[2, 20], edge_attr=[20, 10], y=[1, 15], descriptors=[179])


In [142]:

for index in range(0,len(train_data)):
    train_data[index].descriptors = torch.FloatTensor([train_data_descriptor[index,:]])
    train_data[index].fingerprints = torch.FloatTensor([train_data_fingerprints[index,:]])

for index in range(0,len(test_data)):
    test_data[index].descriptors = torch.FloatTensor([test_data_desriptors[index,:]])
    test_data[index].fingerprints = torch.FloatTensor([test_data_fingerprints[index,:]])

# Example
print(train_data[0])

Data(x=[9, 78], edge_index=[2, 24], edge_attr=[24, 10], y=[1, 15], descriptors=[1, 987], fingerprints=[1, 2048])


Use dataloader

In [143]:
train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=True)

### Main GNN

#### Model for all targets at once

GNN function

In [115]:
data_labels = dataset[50].y.shape[1]
data_features = dataset[50].x.shape[1]
descriptors_features = train_data[0].descriptors.shape[1]
fingerprints_features = train_data[0].fingerprints.shape[1]

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(data_features, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)
        self.conv4 = GATConv(hidden_channels, hidden_channels)
        self.conv5 = GATConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels+descriptors_features+fingerprints_features, 512)
        self.lin2 = Linear(512, 256)
        self.lin3 = Linear(256, data_labels)

    def forward(self, x, edge_index, edge_attr, batch, descriptors, fingerprints): 
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv3(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv4(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv5(x, edge_index, edge_attr)

        #Returns batch-wise graph-level-outputs by averaging node features across the node dimension, so that for a single graph G
        #its output is computed by
        x = global_mean_pool(x, batch) 
        x2 = descriptors
        x3 = fingerprints
        x = torch.cat((x,x2,x3),1)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin2(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin3(x)
        
 
        return x
    

With normalization layers

In [144]:
data_labels = dataset[50].y.shape[1]
data_features = dataset[50].x.shape[1]
descriptors_features = train_data[0].descriptors.shape[1]
fingerprints_features = train_data[0].fingerprints.shape[1]

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(data_features, hidden_channels)
        self.conv1_norm = GraphNorm(hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.conv2_norm = GraphNorm(hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)
        self.conv3_norm = GraphNorm(hidden_channels)
        self.conv4 = GATConv(hidden_channels, hidden_channels)
        self.conv4_norm = GraphNorm(hidden_channels)
        self.conv5 = GATConv(hidden_channels, hidden_channels)
        self.conv5_norm = GraphNorm(hidden_channels)
        self.lin = Linear(hidden_channels+descriptors_features+fingerprints_features, 512)
        self.norm = BatchNorm(hidden_channels+descriptors_features+fingerprints_features)
        self.lin2 = Linear(512, 256)
        self.norm2 = BatchNorm(512)
        self.lin3 = Linear(256, data_labels)

    def forward(self, x, edge_index, edge_attr, batch, descriptors, fingerprints): 
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv1_norm(x)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2_norm(x)
        x = self.conv3(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv3_norm(x)
        x = self.conv4(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv4_norm(x)
        x = self.conv5(x, edge_index, edge_attr)
        x = self.conv5_norm(x)

        #Returns batch-wise graph-level-outputs by averaging node features across the node dimension, so that for a single graph G
        #its output is computed by
        x = global_mean_pool(x, batch) 
        x2 = descriptors
        x3 = fingerprints
        x = torch.cat((x,x2,x3),1)
        x = self.norm(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.norm2(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin2(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin3(x)
        
 
        return x
    

With attention before dense layer (Doesn't work as it is now)

In [84]:
from torch.nn import MultiheadAttention

data_labels = dataset[50].y.shape[1]
data_features = dataset[50].x.shape[1]
descriptors_features = train_data[0].descriptors.shape[1]
fingerprints_features = train_data[0].fingerprints.shape[1]

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(data_features, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)
        self.conv4 = GATConv(hidden_channels, hidden_channels)
        self.conv5 = GATConv(hidden_channels, hidden_channels)
        self.attention = MultiheadAttention(hidden_channels, 1) # input_size, hidden_size, num_heads(number of parallel self-attention heads to use)
        self.attention2 = MultiheadAttention(descriptors_features, 1) 
        self.attention3 = MultiheadAttention(fingerprints_features, 1) 
        self.lin = Linear(hidden_channels+descriptors_features+fingerprints_features, 512)
        self.lin2 = Linear(512, 256)
        self.lin3 = Linear(256, data_labels)

    def forward(self, x, edge_index, edge_attr, batch, descriptors, fingerprints): 
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv3(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv4(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv5(x, edge_index, edge_attr)

        #Returns batch-wise graph-level-outputs by averaging node features across the node dimension, so that for a single graph G
        #its output is computed by
        x = global_mean_pool(x, batch) 
        x,_ = self.attention(x,x,x)
        x2 = descriptors
        x2,_ = self.attention2(x2,x2,x2)
        x3 = fingerprints
        #x3,_ = self.attention3(x3,x3,x3)
        x = torch.cat((x,x2,x3),1)
        #x,_ = self.attention(x,x,x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin2(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin3(x)
        
 
        return x
    

Train GNN

In [145]:
model = GNN(hidden_channels=128) 
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=5e-4)

# Will multiply learning rate by 0.2 every 10 epoch (Haven't tried this yet)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.2) 

criterion = torch.nn.MSELoss()

def train(data_in):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data_in.x, data_in.edge_index, data_in.edge_attr, data_in.batch, data_in.descriptors, data_in.fingerprints)  # Perform a single forward pass.

      #Alt 1
      loss = criterion(out, data_in.y) 

      #Alt 2
      #loss = 0
      #for item in range(0,len(data_in.y[0,:])):
      #      loss += criterion(out[:,item], data_in.y[:,item]) 

      loss.backward()  # Derive gradients.
      # scheduler.step()
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(data):
      all_test_r2 = []
      counter = -1    
      for data_in in data:
            counter += 1
            model.eval()
            
            out = model(data_in.x, data_in.edge_index, data_in.edge_attr, data_in.batch, data_in.descriptors, data_in.fingerprints)

            # Caculate R2
            r2_score_var = []
            for item in range(0,data_in.y.shape[1]):
                  if item == 0:
                        r2_score_var = r2_score(data_in.y[:,item].detach().numpy(), out[:,item].detach().numpy())
                  else:
                        r2_score_var = np.vstack((r2_score_var,(r2_score(data_in.y[:,item].detach().numpy(), out[:,item].detach().numpy()))))

            if counter == 0:
                  all_test_r2 = r2_score_var
            else:
                  all_test_r2 = np.hstack((all_test_r2,r2_score_var))

      average_test_r2 = np.sum(all_test_r2,axis=1) / all_test_r2.shape[1]

      return average_test_r2

# Vectors to append accuracy to:
Train_r2 = []
Test_r2 = []
N = 5 # How often it should calculate R2

# Calculate accuracy before training 
Train_r2.append(test(train_loader))
Test_r2.append(test(test_loader))
print("Initial training R2: ", Train_r2[0])
print("Initial test R2: ", Test_r2[0])

print_r2_option = True
counter = 0
for epoch in range(1, 31):
      counter += 1
      average_loss = []
      for data in train_loader:
            loss = train(data)
            average_loss.append(loss)
      print(f'Epoch: {epoch:03d}, Loss: {(sum(average_loss)/len(average_loss)):.4f}')

      if print_r2_option & (counter % N == 0):

            temp_train_r2 = test(train_loader)
            Train_r2.append(temp_train_r2)

            temp_test_r2 = test(test_loader)
            Test_r2.append(temp_test_r2)

            print(f'Average Train R2: {temp_train_r2}')
            print(f'Average Test R2: {temp_test_r2:}')

Initial training R2:  [-0.02154199 -0.01994992 -0.01134434 -0.0244989  -0.01556203 -0.02195235
 -0.01098373 -0.01563356 -0.01897385 -0.02065857 -0.01878104 -0.02010609
 -0.02260715 -0.02517599 -0.01925064]
Initial test R2:  [-0.02300163 -0.02310306 -0.01319037 -0.02044358 -0.01602927 -0.02159862
 -0.01221507 -0.01462832 -0.02145343 -0.02202356 -0.01905706 -0.02022325
 -0.0251529  -0.02757073 -0.02030301]
Epoch: 001, Loss: 0.1490
Epoch: 002, Loss: 0.1206
Epoch: 003, Loss: 0.1169


KeyboardInterrupt: 

#### Model for just one target

GNN function

GATConv: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GATConv.html#torch_geometric.nn.conv.GATConv

GENConv: https://pytorch-geometric.readthedocs.io/en/latest/generated/torch_geometric.nn.conv.GENConv.html#torch_geometric.nn.conv.GENConv

In [ ]:
data_labels = 1
data_features = dataset[50].x.shape[1]
descriptors_features = train_data[50].descriptors.shape[1]
fingerprints_features = train_data[0].fingerprints.shape[1]

class GNN(torch.nn.Module):
    def __init__(self, hidden_channels):
        super().__init__()
        torch.manual_seed(12345)
        self.conv1 = GATConv(data_features, hidden_channels)
        self.conv2 = GATConv(hidden_channels, hidden_channels)
        self.conv3 = GATConv(hidden_channels, hidden_channels)
        self.conv4 = GATConv(hidden_channels, hidden_channels)
        self.conv5 = GATConv(hidden_channels, hidden_channels)
        self.lin = Linear(hidden_channels+descriptors_features+fingerprints_features, 512)
        self.lin2 = Linear(512, 256)
        self.lin3 = Linear(256, data_labels)

    def forward(self, x, edge_index, edge_attr, batch, descriptors, fingerprints): 
        x = self.conv1(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv2(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv3(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv4(x, edge_index, edge_attr)
        x = F.relu(x)
        x = self.conv5(x, edge_index, edge_attr)

        #Returns batch-wise graph-level-outputs by averaging node features across the node dimension, so that for a single graph G
        #its output is computed by
        x = global_mean_pool(x, batch) 
        x2 = descriptors
        x3 = fingerprints
        x = torch.cat((x,x2,x3),1)
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.lin(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin2(x)
        x = F.dropout(x, p=0.2, training=self.training)
        x = F.relu(x)
        x = self.lin3(x)
        
 
        return x
    

Train GNN

In [ ]:
def train(data_in, target):
      model.train()
      optimizer.zero_grad()  # Clear gradients.
      out = model(data_in.x, data_in.edge_index, data_in.edge_attr, data_in.batch, data_in.descriptors, data_in.fingerprints)
      targets = data_in.y[:,target].reshape(-1,1)
      
      #Alt 1
      loss = criterion(out, targets)   

      loss.backward()  # Derive gradients.
      optimizer.step()  # Update parameters based on gradients.
      return loss

def test(data, target):
      all_test_r2 = []
      counter = -1    
      for data_in in data:
            counter += 1
            model.eval()
            out = model(data_in.x, data_in.edge_index, data_in.edge_attr, data_in.batch, data_in.descriptors, data_in.fingerprints)
            targets = data_in.y[:,target].reshape(-1,1)
            
            # Caculate R2
            r2_score_var = r2_score(targets.detach().numpy(), out.detach().numpy())

            all_test_r2 .append(r2_score_var)

      average_test_r2 = np.sum(all_test_r2) / len(all_test_r2)

      return average_test_r2

num_targets = dataset[50].y.shape[1]
for target_index in range(num_targets):
      print("Target index: ", target_index)

      model = GNN(hidden_channels=64) 
      optimizer = torch.optim.Adam(model.parameters(), lr=0.0001, weight_decay=5e-4)
      criterion = torch.nn.MSELoss()

      # Vectors to append accuracy to:
      Train_r2 = []
      Test_r2 = []

      # Calculate accuracy before training 
      Train_r2.append(test(train_loader, target_index))
      Test_r2.append(test(test_loader, target_index))
      print("Initial training R2: ", Train_r2[0])
      print("Initial test R2: ", Test_r2[0])

      print_r2_option = True
      for epoch in range(1, 21):
            average_loss = []
            for data in train_loader:
                  loss = train(data, target_index)
                  average_loss.append(loss)
            print(f'Epoch: {epoch:03d}, Loss: {(sum(average_loss)/len(average_loss)):.5f}')

            if print_r2_option:
                  temp_train_r2 = test(train_loader, target_index)
                  Train_r2.append(temp_train_r2)

                  temp_test_r2 = test(test_loader, target_index)
                  Test_r2.append(temp_test_r2)

                  # print(f'Average Train R2: {temp_train_r2}')
                  # print(f'Average Test R2: {temp_test_r2:}')

      print(f"Best training R2 for target {target_index}: {np.max(Train_r2)}")
      print(f"Best test R2 for target {target_index}: {np.max(Test_r2)}")

Target index:  0
Initial training R2:  -0.018678295383913163
Initial test R2:  -0.03308373704488183
Epoch: 001, Loss: 0.76507
Epoch: 002, Loss: 0.63190
Epoch: 003, Loss: 0.56252
Epoch: 004, Loss: 0.51782
Epoch: 005, Loss: 0.48434
Epoch: 006, Loss: 0.46161
Epoch: 007, Loss: 0.44506
Epoch: 008, Loss: 0.43061
Epoch: 009, Loss: 0.41689
Epoch: 010, Loss: 0.40696
Epoch: 011, Loss: 0.39623
Epoch: 012, Loss: 0.38796
Epoch: 013, Loss: 0.37597
Epoch: 014, Loss: 0.37060
Epoch: 015, Loss: 0.36610
Epoch: 016, Loss: 0.35919
Epoch: 017, Loss: 0.35353
Epoch: 018, Loss: 0.34897
Epoch: 019, Loss: 0.34382
Epoch: 020, Loss: 0.34009
Best training R2 for target 0: 0.6581509149316066
Best test R2 for target 0: 0.6554282682601079
Target index:  1
Initial training R2:  -0.01632305111746684
Initial test R2:  -0.017224460553526586
Epoch: 001, Loss: 0.63536
Epoch: 002, Loss: 0.51249
Epoch: 003, Loss: 0.45557
Epoch: 004, Loss: 0.41914
Epoch: 005, Loss: 0.40053
Epoch: 006, Loss: 0.38774
Epoch: 007, Loss: 0.37415
Ep

## Transformer